# Рекомендательная система "Подбор объектов недвижимости"

Набор данных Bangalore House Price


- AREA_TYPE : тип района, где расположен объект недвижимости.
- AVAILABILITY : доступна ли недвижимость в настоящее время или нет.
- LOCATION : Место, где находится недвижимость.
- SIZE : Количество спален
- SOCIETY : Тип района, в котором находится собственность.
- TOTAL_SQFT : общая площадь объекта в квадратных футах.
- BATH : Количество ванных комнат
- BALCONY : Количество балконов
- PRICE : Стоимость недвижимости в лакхах¶

In [10]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from joblib import dump
import difflib
pd.set_option('future.no_silent_downcasting', True)

In [7]:
class ML_MODEL():
    def __init__(self,filename):
        self.filename=filename
        # Читаем набор данных
        self.df = pd.read_csv(filename)

    def get_clean_data(self):
        # Данный метод:
        # 1. Читает набор данных в csv
        # 2. Выполняет очистку данных, чтобы не было пропусков и нечисловых полей
        # Данное поле содержит очень много пропусков, удаляем
        self.df.drop('society',axis=1,inplace=True)
        # Заменяем пропуски в данных
        self.df['location']=self.df['location'].replace(to_replace=np.nan, value=self.df.location.mode()[0])

        self.df['area_type']=self.df['area_type'].replace(to_replace=np.nan, value=self.df.area_type.mode()[0])
        self.df['size']=self.df['size'].str.split(' ',expand=True)[0]
        self.df['size'] = self.df['size'].fillna(1)
        self.df['bath'] = self.df['bath'].fillna(1)
        self.df['balcony'] = self.df['balcony'].fillna(0)
        self.df.availability=(self.df.availability=='Ready To Move')
        self.df['availability']=self.df['availability'].replace({True:1,False:0})

        #Заменяем пропуски в данных на основе интерполяции
        self.df['total_sqft'] = self.df['total_sqft'].interpolate(method='polynomial', order=2)
        self.df['price'] = self.df['price'].interpolate(method='polynomial', order=2)
        # на этом шаге сохраним набор очищенный набор данных
        # Далее заменяем категориальные данные на числовые коды
        encoder = {}
        encoder["area_type"]=LabelEncoder().fit(self.df['area_type'])
        encoder['location']=LabelEncoder().fit(self.df['location'])
        self.df['area_type'] = encoder["area_type"].transform(self.df['area_type'])
        self.df['location'] = encoder["location"].transform(self.df['location'])
        return self.df, encoder

    def build_model(self):
        # Строим модель обучения по методу ближайших соседей
        # Сохраняем скомпилированную модель в файл
        self.model = NearestNeighbors(metric='cosine')
        self.model.fit(self.df)
        return self.model


In [34]:
class VIEW():
    def __init__(self,data,encoder):
        self.df=data
        self.encoder=encoder

    def get_simp(self, s, lst):
        #Выбирает название из списка максимально похожее на ввод пользователя
        normalized1 = s.lower()
        mratio=0
        simp=0
        for k in range(len(lst)):
        # Нормализуем элемент списка значений категории
            normalized2 = lst[k].lower()
        # Определяем близость текущего элемента списка значений категории и ввода пользователя
            matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
            r= matcher.ratio()
        # Ищем максимально близкий элемент списка значений категории (с самым высоким рейтингом)
            if r>mratio:
                mratio=r
                simp=k
        return simp

    def get_user_data(self):
        X = {'area_type': 0,
             'availability': 1,
             'location': 300,
             'size': 2,
             'total_sqft': 300,
             'bath': 1,
             'balcony': 1,
             'price': 300}
        print("Введите данные для подбора объекта недвижимости:")

        print('Тип объекта недвижимости')
        lst=self.df['area_type'].unique()
        lst=self.encoder['area_type'].inverse_transform(self.df['area_type'].unique())
        print('У нас в базе есть следующие типы:\n',"\n".join(lst))
        choose=input("Введите тип объекта недвижимости: ")
        X['area_type']=self.get_simp(choose,lst)

        return X

    def get_objects_number(self):
        choose=int(input('Введите количество объектов в рекомендации: '))
        return choose

    def print_recomendation(self,lst):
        print('Вашему запросу соответствуют следующие объекты недвижимости:')
        for x in lst:
            print("---------",x,"------------")
            res=self.df.loc[x,:].copy()
            res['area_type']=self.encoder['area_type'].inverse_transform([res['area_type']])
            print(res)


In [4]:
class BOT():
    def __init__(self,filename):
        self.csv_data=filename
        self.clean_data=""
        self.dm=""
        self.model=""
        self.encoder=""


    def create_model(self):
        self.dm = ML_MODEL(self.csv_data)
        self.clean_data, self.encoder=self.dm.get_clean_data()
        self.model=self.dm.build_model()

    def get_recommentation(self):
        #Создаем экземпляр view на основе очищенного набора данных
        view=VIEW(self.clean_data, self.encoder)
        # Читаем ввод пользователя
        X=view.get_user_data()
        # Создаем набор данных для построения рекомендации
        x = pd.DataFrame(columns=X.keys())
        x.loc[0]=X.values()
        # Вводим количество объектов в рекомендации
        k=view.get_objects_number()
        # Строим рекомендацию
        y = self.model.kneighbors(x, return_distance=False, n_neighbors=k)
        # Выводим найденные объекты на экран
        view.print_recomendation(y[0])

In [35]:
bot=BOT("https://raw.githubusercontent.com/yakushinav/mgimo_2/refs/heads/main/Bengaluru_House_Data.csv")
bot.create_model()
bot.get_recommentation()

Введите данные для подбора объекта недвижимости:
Тип объекта недвижимости
У нас в базе есть следующие типы:
 Super built-up  Area
Plot  Area
Built-up  Area
Carpet  Area
Введите тип объекта недвижимости: Plot
Введите количество объектов в рекомендации: 3
Вашему запросу соответствуют следующие объекты недвижимости:
--------- 9193 ------------
area_type       Super built-up  Area
availability                       0
location                        1249
size                               3
total_sqft                    1496.0
bath                             2.0
balcony                          1.0
price                          718.0
Name: 9193, dtype: object
--------- 7881 ------------
area_type       Super built-up  Area
availability                       1
location                        1191
size                               4
total_sqft                    2000.0
bath                             3.0
balcony                          2.0
price                         1063.0
Name: 7881,